In [1]:
import numpy as np

from tqdm.notebook import tqdm

In [2]:
def np_dot(a, b):
  return np.dot(a,b)

def norm(a):
  return np.linalg.norm(a)

In [3]:
def func(x):
  """
  Функция произвольного
  вида от вектора х.
  """

  x = np.asarray(x).reshape(-1)
  A = np.eye((x.shape[0]))
  b = np.ones_like(x)
  c = 0
  
  return np_dot(np_dot(A,x),x) + np_dot(b,x) + c

In [5]:
def get_grad(functor, x_0):
  """
  Принимаем на вход функтор и
  координаты точки x_0,
  возвращаем градиент 
  (вектор столбец)
  в этой точке.
  """
  
  eps = 1e-9
  x_0 = np.asarray(x_0, dtype=float).reshape(-1)
  f_grad = np.zeros_like(x_0)

  for i in range(x_0.shape[0]):
    x_plus = x_0.copy()
    x_plus[i] += eps

    x_minus = x_0.copy()
    x_minus[i] -= eps

    f_grad[i] = functor(x_plus) - functor(x_minus)
    f_grad[i] /= 2*eps

  return f_grad.reshape(-1,1)

In [6]:
import time

In [50]:
def DFP_method(func, x, eps=1e-6, logging=False):

  # Help function for leraning rate search.
  #
  def one_dim_min(f, x_cur, d_cur):
    """
    One-dimensional minimization rule.

    Ищем lr с помощью градиентного спуска 
     с шагом lrlr, который уменьшается со временем.
    """

    lr = 0.5
    lrlr = 0.1
    f_from_lr = lambda lr: f(x_cur - lr * d_cur)
    fx_prev = np_dot(get_grad(f_from_lr, lr).T, lr) # <f',x>

    for _ in range(1000):          
      th = 1
      lr_d_cur = np.clip(get_grad(f_from_lr, lr), -th, th)[0,0]
      lr = lr - lrlr * lr_d_cur
      fx_next = np_dot(get_grad(f_from_lr, lr).T, lr)
      if fx_next * fx_prev > 0: # Если направление движения не меняется увеличиваем шаг, иначе уменьшаем.
        lrlr *= 1.2
      else:
        lrlr *= 0.6
      fx_prev = fx_next
      # print()
      # print('lr: %.4f' % lr)
      # print('norm lr_d: %.12f' % norm(lr_d_cur))
      # print(eps)
      # print("f\' = %.2f" % fx_next)
      # print('lrlr: %.4f' % lrlr)
      if norm(lr_d_cur) < eps:
        break
      #time.sleep(4)

    return lr


  x = np.asarray(x).reshape(-1,1)

  if logging:
    i = 1

  while norm(get_grad(func, x)) > eps:    

    if logging:
      print('%i-st step of DFP:' % i)
    i+=1

    if logging:
      print("f(x) = %.6f" % func(x))
      #print("f\' = %.6f" % np_dot(get_grad(func, x).T, x))
      print("||f\'|| = %.6f" % norm(get_grad(func, x)))

    Q = np.diag([1.]*x.shape[0])

    x_prev = x
    f_grad_prev = get_grad(func, x_prev)

    d = -np_dot(Q, f_grad_prev)
    lr = one_dim_min(func, x_prev, f_grad_prev) # Ищем длину шага.
    if logging:
      print('lr = %.4f' % lr)
    x_next = x_prev + lr * d  
    f_grad_next = get_grad(func, x_next)

    if logging:
      print('<f\'next, f\'prev> = %.3f' % np_dot(f_grad_next.T, f_grad_prev)) # Проверяем оротогональны ли напр-ия.

    r = x_next - x_prev
    s = f_grad_next - f_grad_prev

    Q_s = np_dot(Q, s)
    Q += np_dot(r, r.T) / np_dot(r.T, s)
    Q -= np_dot(Q_s, Q_s.T) / np_dot(Q_s, s.T)

    x = x_next    

    if logging:
      print('step has been taken', end='\n\n')
  
  if logging:
    print('DFP method converged.', end='\n\n')

  return x  

In [51]:
x_0 = [-5, 10]
x = DFP_method(func, x_0, eps=1e-6, logging=True)
print(func(x), '\n', x)

1-st step of DFP:
f(x) = 130.000000
||f'|| = 22.847331
lr = 0.5000
<f'next, f'prev> = 0.000
step has been taken

2-st step of DFP:
f(x) = -0.500000
||f'|| = 0.000015
lr = 0.5000
<f'next, f'prev> = 0.000
step has been taken

DFP method converged.

-0.49999999999999867 
 [[-0.50000002]
 [-0.50000003]]


In [52]:
def g_1(x):

  x = np.asarray(x).reshape(2,1)
  const = np.array([1,1]).reshape(2,1)

  return np.float(np_dot(const.T, x))

In [53]:
def g_2(x):

  x = np.asarray(x).reshape(2,1)
  const = np.array([1,-1]).reshape(2,1)

  return np.float(np_dot(const.T, x))

In [54]:
def f(x):

  x = np.asarray(x).reshape(2,1)
  const = np.array([10, 9]).reshape(2,1)

  x = x + const
  return np.float(np_dot(x.T, x))

In [33]:
# def Z_func(x, f, G, r):

#   x = np.asarray(x).reshape(-1,1)
#   p = 10
  
#   rest = 0
#   for i in range(x.shape[0]):
#     rest += np.max(0, -G[i](x))**p

#   return f(x) + r * rest

In [55]:
def Find_optim(func, x, g_list = [], h_list = [], p=2, r=1, beta=2, eps=1e-6, logging=False):
  """
  x  - начальная точка.
  func - функция, минимум которой мы ищем.
  g_list - список функций-ограничений типа нер-во (>= 0).
  h_list - список функций-ограничений типа равенство.
  p - показатель степени (см. Z_func).
  r - начальное значение параметра штрафа.
  beta - коэф-нт в формуле вычисления параметра штрафа.
  eps  - условия останова, ограничение на ||f'||.
  logging - выводить логи работы или нет.

  return точку x_optim, в кот-ой достигается оптимум func.
  """


  # 2-nd part in Z_func, func - 1st.
  #
  def alpha_func(x, p=p): 

    alpha_func_val = 0    
    for g_func in g_list:
      alpha_func_val += max(0, - g_func(x)) ** p  

    for h_func in h_list:
      alpha_func_val += abs(h_list(x)) ** p
    
    return alpha_func_val

  
  # Main function for finding the optimum.
  #
  def Z_func(x):
    return func(x) + r * alpha_func(x)

  # Function for logging.
  #
  def barriers_logging(x, r):
    print('curr Z_val: %.5f, from' % Z_func(x))
    print('curr optim point: ', end='')
    for x_coor in x: print('%.3f  ' % x_coor, end='')
    print('\nr = %i' % r)
    print('alpha(x) = %.6f' % alpha_func(x))
    print('-'*30)

  if logging:
    print('Search has started!')
    print('-'*30)

  x = DFP_method(Z_func, x, logging=logging)

  if logging:
    barriers_logging(x, r)

  while r*alpha_func(x) > eps:

    r *= beta
    x = DFP_method(Z_func, x, eps, logging=logging)

    if logging:
      barriers_logging(x, r)
    
  if logging:
    print('Search has ended!')
  
  return x

In [56]:
%%time
x_0 = np.array([[2],[-1]])

x_optim = Find_optim(func, x_0, [g_1, g_2], eps=1e-6, r=1, logging=True)
x_optim

Выходные данные были обрезаны до нескольких последних строк (5000).
||f'|| = 0.001188
lr = 0.2572
<f'next, f'prev> = -0.000
step has been taken

726-st step of DFP:
f(x) = -0.166667
||f'|| = 0.000645
lr = 0.5000
<f'next, f'prev> = -0.000
step has been taken

727-st step of DFP:
f(x) = -0.166667
||f'|| = 0.001291
lr = 0.2362
<f'next, f'prev> = -0.000
step has been taken

728-st step of DFP:
f(x) = -0.166667
||f'|| = 0.000538
lr = 0.5000
<f'next, f'prev> = -0.000
step has been taken

729-st step of DFP:
f(x) = -0.166667
||f'|| = 0.001077
lr = 0.2824
<f'next, f'prev> = -0.000
step has been taken

730-st step of DFP:
f(x) = -0.166667
||f'|| = 0.000748
lr = 0.4643
<f'next, f'prev> = -0.000
step has been taken

731-st step of DFP:
f(x) = -0.166667
||f'|| = 0.001335
lr = 0.2284
<f'next, f'prev> = -0.000
step has been taken

732-st step of DFP:
f(x) = -0.166667
||f'|| = 0.000494
lr = 0.5000
<f'next, f'prev> = -0.000
step has been taken

733-st step of DFP:
f(x) = -0.166667
||f'|| = 0.000988
lr

KeyboardInterrupt: ignored